## **Extracting dataset from Zillow**

The first step in any machine learning project is to obtain a dataset. In this case, we will use Zillow's GetSearchPageState request in order to retrieve the search results of a particular query. 

The initial page shown when looking for real estate listings on Zillow, shows all of the listings on a map, or a menu with multiple pages. In this case, we are looking for all of the homes sold in San Diego County in the last 12 months (April 2022 - April 2023.)

![Zillow Page](images/zillow_1.PNG)

We can extract the listings that match our query by using the request that Zillow itself uses to retrieve the data, called GetSearchPageState. We can acess this request by opening the browser's developer tools, and looking at the Fetch/XHR network requests. We find that a GetSearchPageState request is made every time we change the parameters of the query (map size, price range, pagination, etc.)

![Zillow Page](images/zillow_2.PNG)

We can look at a preview of the request response, which returns a nested dictionary of all of the data retrieved by the request. This response contains information about the number of listings, the number of pages, and most importantly, the actual data of the listings. 

![Zillow Page](images/zillow_3.PNG)
![Zillow Page](images/zillow_4.PNG)
![Zillow Page](images/zillow_5.PNG)

As we can see, the listings are held in a list inside the request response. We will use Python to make the requests that extract all (or most) of our data in order to build a dataset directly from Zillow's listings.

We start off by copying the request as a Bash(cURL) request and use an online tool that will convert our request to the Python language. We get a dictionary of request headers, which are constant for every request made using Zillow, and a request link, whose response will be obtained using the .get method from the Python requests module. 

![Zillow Page](images/zillow_6.PNG)

We can move on to implementing the Python code. First, we import the modules that are going to be needed. We will use requests to make the requests and retrieve the responses, and pandas to save the data from the listings to a dataframe.

In [1]:
# Import required modules
import numpy as np
import pandas as pd
import requests
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

Since the request headers are all constant between links from a common website, we can use them for all of the requests we are going to make. 

In [2]:
# Define the request headers (taken from Zillow GetSearchPageState API request)
headers = {
    'authority': 'www.zillow.com',
    'accept': '*/*',
    'accept-language': 'es-ES,es;q=0.9,en-US;q=0.8,en;q=0.7',
    'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
}

The GetSearchPageState API request can give a maximum of 500 results from the map, and 40 results for every page, but it was found that a lot of listings were lost when using the map results, so we will define a search query, extract the results from one page, then move over to the next page and so on, until we get all the results associated with the query. 
The get_data function accepts a request URL associated with certain parameters, extracts the number of pages and iterates through every page in order to retrieve all of the listings. 

In [3]:
# Function that takes in a request URL and returns a list of dictionaries with the data for the Zillow listings from each result page
def get_data(url):
    response = requests.get(url, headers=headers)
    pages = response.json()['cat1']['searchList']['totalPages']
    data = []
    split = url.split('%22currentPage%22%3A2')
    for page in range(1, pages+1):
        page_url = split[0] + '%22currentPage%22%3A' + str(page) + split[1]
        response = requests.get(page_url, headers=headers)
        data.extend(response.json()['cat1']['searchResults']['listResults'])
    return data

Just as the map results, the list results on Zillow are also limited to a maximum of 40 results per page, at 20 pages max for a maximum of 800 listings per query. We will use price ranges in order to reduce the number of listings associated with each query so that we can extract as much data as possible. Below, we make a list of all request links of listings with prices ranging from \$0 to \$20,000,000, and we run each link through our page iterating function in order to obtain a list with the information of every house found in Zillow that was sold in the last 12 months in San Diego County.

In [ ]:
# Defining the bins for the price range
bins = [i * 20000 for i in range(0, 1001)]

# Generating the request URLs for each price range (to mitigate the 500 maximum results per page limit)
links = ['https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState=%7B%22mapBounds%22%3A%7B%22north%22%3A33.97337635760156%2C%22east%22%3A-115.66360693554688%2C%22south%22%3A32.052726223972044%2C%22west%22%3A-118.02841406445313%7D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22doz%22%3A%7B%22value%22%3A%2212m%22%7D%2C%22sortSelection%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22isAllHomes%22%3A%7B%22value%22%3Atrue%7D%2C%22isRecentlySold%22%3A%7B%22value%22%3Atrue%7D%2C%22isForSaleByAgent%22%3A%7B%22value%22%3Afalse%7D%2C%22isForSaleByOwner%22%3A%7B%22value%22%3Afalse%7D%2C%22isNewConstruction%22%3A%7B%22value%22%3Afalse%7D%2C%22isComingSoon%22%3A%7B%22value%22%3Afalse%7D%2C%22isAuction%22%3A%7B%22value%22%3Afalse%7D%2C%22isForSaleForeclosure%22%3A%7B%22value%22%3Afalse%7D%2C%22price%22%3A%7B%22min%22%3A' + str(bins[i+1]) + '%2C%22min%22%3A' + str(bins[i]) +'%7D%2C%22monthlyPayment%22%3A%7B%22min%22%3A494%2C%22max%22%3A987%7D%7D%2C%22isListVisible%22%3Atrue%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A2841%2C%22regionType%22%3A4%7D%5D%2C%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22mapZoom%22%3A9%7D&wants={%22cat1%22:[%22listResults%22,%22mapResults%22]}&requestId=12' for i in range(len(bins)-1)]

# Extracting the data of all listings
import tqdm
listings = []
for link in tqdm.tqdm(links):
    listings.extend(get_data(link))

Now, we iterate though every listing, extract the useful information and append it to a dataframe as a new entry. 

In [ ]:
# Extract the important information from the listings and converting to pandas dataframe
df = pd.DataFrame()
for listing in tqdm.tqdm(listings):
    if 'hdpData' in listing:
        information = listing['hdpData']['homeInfo']
        df = pd.concat([df, pd.DataFrame(information, index=[0])], axis=0)

Finally, we clean our data by removing duplicate listings, resetting the index and saving it as a .csv file so that it can be loaded without making the requests a second time.

In [ ]:
# Clean the data and save to .csv
df = df.drop_duplicates(subset=['zpid'])
df.reset_index(drop = True, inplace=True)
df.to_csv('zillow_listings.csv', index=False)

Now that we saved the dataframe, we can visualize the first 5 entries of our dataset, which consists of 26701 listings with 44 features each.

In [9]:
# Visualize the data
pd.set_option('display.max_columns', None)
df.head(5)

,zpid,streetAddress,zipcode,city,state,latitude,longitude,price,dateSold,bathrooms,bedrooms,livingArea,homeType,homeStatus,daysOnZillow,isFeatured,shouldHighlight,zestimate,rentZestimate,listing_sub_type,isUnmappable,isPreforeclosureAuction,homeStatusForHDP,priceForHDP,isNonOwnerOccupied,isPremierBuilder,isZillowOwned,currency,country,taxAssessedValue,lotAreaValue,lotAreaUnit,unit,isRentalWithBasePrice,datePriceChanged,priceReduction,priceChange,videoCount,providerListingID,newConstructionType,contingentListingType,openHouse,open_house_info,comingSoonOnMarketDate
0,16765112,2330 Via Aprilia,92014,Del Mar,CA,32.934574,-117.251900,8500.0,1.680246e+12,4.0,3.0,2899.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,2759800.0,9650.0,NaN,False,False,RECENTLY_SOLD,8500.0,True,False,False,USD,USA,1340151.0,7840.00,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,16731749,16938 Via De Santa Fe,92091,Rancho Santa Fe,CA,33.018356,-117.201935,6499.0,1.677917e+12,2.0,2.0,1530.0,CONDO,RECENTLY_SOLD,-1,False,False,1359200.0,3898.0,NaN,False,False,RECENTLY_SOLD,6499.0,True,False,False,USD,USA,557001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,82941077,260 E Bradley Ave SPACE 29,92021,El cajon,CA,32.821266,-116.958670,20000.0,1.677139e+12,1.0,2.0,800.0,MANUFACTURED,RECENTLY_SOLD,-1,False,False,NaN,2281.0,NaN,False,False,RECENTLY_SOLD,20000.0,True,False,False,USD,USA,24248.0,2.78,acres,Space 29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,61238050,220 Lyon Cir,92083,Vista,CA,33.192005,-117.238144,3895.0,1.676880e+12,2.5,3.0,1903.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,828800.0,3792.0,NaN,False,False,RECENTLY_SOLD,3895.0,True,False,False,USD,USA,569249.0,9741.00,sqft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16875584,9356 Las Lomas Dr,92071,Santee,CA,32.863760,-117.001550,1000.0,1.675843e+12,2.0,3.0,1676.0,SINGLE_FAMILY,RECENTLY_SOLD,-1,False,False,893100.0,3800.0,NaN,False,False,RECENTLY_SOLD,1000.0,True,False,False,USD,USA,795000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In the next steps of the project, we will use this dataset to visualize the patterns in the data, and build a model that predicts the price based on common variables in the data.